# 04 ポップコーン

### メモ  
GAOZで完全一致しないやつがある  
->ハミング距離最小のやつで絞る

### 担当者と日付
2024/10/1 渡邉健  
2024/10/2 ごとう

## 準備

### ライブラリのインポート  

In [384]:
import pandas as pd
import numpy as np
import warnings
import constants as cons #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)

### C_0 ~ C_9までの読み込み

In [385]:
from os import listdir

ATTACK_TARGET = "04"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,56,6,977,3,0,2,4,0,4,2,3,0,1,0,2,0
1,M,18,19,934,5,3,0,0,4,3,1,2,2,0,0,5,3
2,M,35,14,794,0,3,2,4,4,0,0,5,3,1,4,1,0
3,F,56,0,550,3,2,1,4,1,1,3,5,0,4,4,4,1
4,M,50,10,186,1,5,0,0,3,2,5,1,3,5,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,18,17,284,2,2,0,4,4,0,5,2,1,0,0,1,4
9996,F,18,16,503,2,3,0,2,1,0,1,0,5,2,3,3,2
9997,M,35,3,494,4,3,0,0,0,4,3,0,2,0,1,3,0
9998,F,18,17,128,4,2,0,1,4,4,3,5,0,2,0,4,0


### Ba, Bbの読み込み

In [386]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=cons.ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Ximenes Huxtable,M,1,10,296,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Catlaina Dobby,F,35,1,919,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Audi Steggals,F,18,12,371,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yuma Methley,M,25,17,224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Rafael Cubley,M,1,1,495,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [387]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,1,4,4,2,0,5,3,1,1,*,...,0,0,3,0,2,3,3,3,0,1
1,3,3,4,1,4,4,4,4,4,1,...,0,4,0,4,2,4,0,4,2,2
2,2,2,0,5,3,0,0,0,0,4,...,0,0,4,0,0,0,4,0,0,0
3,1,2,0,3,0,0,0,5,*,1,...,0,2,5,4,2,4,0,2,1,0
4,5,4,1,4,3,5,4,1,3,3,...,2,0,2,5,4,4,1,0,1,2


## 分析

### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [388]:
# 試しにBaの3行目を選択
N = 3

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

filtered_C_list = []


検索条件
M 25 17 224


In [389]:

# # 完全一致を見つける
# num = 0
# for c in c_data_list:
#     filtered_row = c[(c['Gender'] == gender) & 
#                       (c['Age'] == int(age)) & 
#                       (c['Occupation'] == int(occupation)) & 
#                       (c['ZIP-code'] == int(zipcode))]
#     if not filtered_row.empty:
#         num += 1
#         filtered_C_list.append(filtered_row)

# print("完全一致した件数 (10であることが望まれる)")
# print(num)
# filtered_C_list


### GAOZが完全一致するものでなく、各Ciの中でHamming distが最小のものをリスト化する

In [390]:
print(c_data_list[0].iloc[1].iloc[0])
print(c_data_list[0].iloc[1].iloc[1])
print(c_data_list[0].iloc[1].iloc[2])
print(c_data_list[0].iloc[1].iloc[3])

M
18
19
934


In [391]:
#あるrowのhamming dist
base = [gender, age, occupation, zipcode]
# print(base)
def GAOZ_hamming(comp):
    ham = 0
    if comp['Gender'] != gender:
        ham += 1
    if comp['Age'] != age:
        ham += 2
    if  comp['Occupation'] != occupation:
        ham += 2
    if comp['ZIP-code'] != zipcode:
        ham += 3

    return ham

res = []
hams = []
index = 0
for cf in c_data_list:
    # print(index, end = '')
    #cf = c_data_list[0]
    min = 100
    cand = []
    for i in range(0,10000):
        tmp = GAOZ_hamming(cf.iloc[i])
        if tmp == min:
            cand.append(cf.iloc[i])
        if tmp < min:
            # print(' ', tmp)
            min = tmp
            cand = [cf.iloc[i]]
    # print(' ',min)
    hams.append(min)
    for elem in cand:
        res.append(pd.DataFrame(elem))
    index += 1
print(hams)

filtered_C_list = res
# for elem in filtered_C_list:
#     for i in range(0,4):
#         print(elem.iloc[i].iloc[0],',', end = '')
#     print()
print(len(filtered_C_list))

[0, 2, 0, 2, 2, 2, 0, 0, 2, 2]
26


In [392]:
filtered_C_list[0].iloc[0].iloc[0]

'M'

### 各カラムごとに多数決の候補を表示する

In [393]:
merged_rows = []

#見やすさのために、大元に含まれてるカラムを補完
for i, filtered_df in enumerate(filtered_C_list):
    print(filtered_df.empty)
    if not filtered_df.empty:
        # GAOZが完全一致してものがCiの中に複数あった場合のために各フィルタリングされたデータフレームから1行だけ取得
        row = filtered_df
        # print(row)
        
        # ALL_HEADERに基づいて新しい行を作成
        new_row = pd.Series(index=cons.ALL_HEADER)
        
        # データを転送
        for col in cons.ALL_HEADER:
            if col in filtered_df.columns:
                new_row[col] = row[col]
                print('hahahaha')
            else:
                new_row[col] = np.nan
        # print(new_row)
        # 新しい行をリストに追加
        merged_rows.append(new_row)
# print(merged_rows)
# リストからデータフレームを作成
print('merge')
merged_df = pd.DataFrame(merged_rows, columns=cons.ALL_HEADER)
merged_df

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
merge


,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### データの再現　　
各行に対してランダムに一つ値を取得してみる

In [394]:
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
random_values = {}
for column in merged_df.columns:
    random_values[column] = select_random_value(merged_df[column])

# 結果を新しいデータフレームとして作成
result_df = pd.DataFrame([random_values])
result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [395]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=cons.ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = Ba.iloc[row_index]["Age"]
    occupation = Ba.iloc[row_index]["Occupation"]
    zipcode = Ba.iloc[row_index]["ZIP-code"]

    filtered_C_list = []

    # 各cデータフレームに対して、完全一致するレコードをフィルタリング
    for c in c_data_list:
        filtered_C_0 = c[(c['Gender'] == gender) & 
                        (c['Age'] == int(age)) & 
                        (c['Occupation'] == int(occupation)) & 
                        (c['ZIP-code'] == int(zipcode))]
        filtered_C_list.append(filtered_C_0)

    # 多数決の候補となる行のリストを作成
    candidate_rows = []
    for i, filtered_df in enumerate(filtered_C_list):
        if not filtered_df.empty:
            # 各フィルタリングされたデータフレームから1行だけ取得
            row = filtered_df.iloc[0]
            # ALL_HEADERに基づいて新しい行を作成し、データを転送
            candidate_row = pd.Series(index=cons.ALL_HEADER)
            for col in cons.ALL_HEADER:
                candidate_row[col] = row[col] if col in filtered_df.columns else np.nan
            # 新しい行を候補リストに追加
            candidate_rows.append(candidate_row)

    # 候補行のリストからデータフレームを作成
    candidate_rows_df = pd.DataFrame(candidate_rows, columns=cons.ALL_HEADER)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in cons.ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,1,10,296,NaN,NaN,3.0,0.0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,F,35,1,919,NaN,NaN,NaN,0.0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,F,18,12,371,4.0,0.0,NaN,4.0,0.0,...,0.0,2.0,NaN,NaN,NaN,3.0,2.0,5.0,3.0,4.0
3,NaN,M,25,17,224,NaN,NaN,0.0,0.0,1.0,...,4.0,NaN,0.0,0.0,5.0,NaN,4.0,NaN,0.0,5.0
4,NaN,M,1,1,495,3.0,0.0,0.0,3.0,0.0,...,2.0,NaN,5.0,2.0,0.0,NaN,1.0,2.0,3.0,2.0
5,NaN,F,35,20,459,0.0,4.0,NaN,2.0,1.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,F,25,5,431,NaN,NaN,NaN,3.0,3.0,...,NaN,1.0,2.0,5.0,2.0,0.0,NaN,NaN,5.0,5.0
8,NaN,F,25,5,118,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
9,NaN,F,18,0,65,NaN,NaN,5.0,1.0,2.0,...,NaN,NaN,1.0,3.0,0.0,NaN,NaN,NaN,1.0,0.0


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [396]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[cons.MOVIE_IDS], bb_row[cons.MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[cons.MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[cons.MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,1,10,296,NaN,NaN,3.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,35
1,NaN,F,35,1,919,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,45
2,NaN,F,18,12,371,4.0,0.0,NaN,4.0,0.0,...,NaN,NaN,NaN,3.0,2.0,5.0,3.0,4.0,26,35
3,NaN,M,25,17,224,NaN,NaN,0.0,0.0,1.0,...,0.0,0.0,5.0,NaN,4.0,NaN,0.0,5.0,18,45
4,NaN,M,1,1,495,3.0,0.0,0.0,3.0,0.0,...,5.0,2.0,0.0,NaN,1.0,2.0,3.0,2.0,19,40
5,NaN,F,35,20,459,0.0,4.0,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,2.0,2.0,0.0,0.0,15,45
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,35
7,NaN,F,25,5,431,NaN,NaN,NaN,3.0,3.0,...,2.0,5.0,2.0,0.0,NaN,NaN,5.0,5.0,18,35
8,NaN,F,25,5,118,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,10,35
9,NaN,F,18,0,65,NaN,NaN,5.0,1.0,2.0,...,1.0,3.0,0.0,NaN,NaN,NaN,1.0,0.0,16,35
